# Hybrid

### Import libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
import os
import csv

from utilities import load_sparse_matrix

### Load dataset

In [2]:
URM = load_sparse_matrix("urm.npz")
URM_train = load_sparse_matrix("urm_train.npz")
URM_validation = load_sparse_matrix("urm_val.npz")
ICM = load_sparse_matrix("icm.npz")

Loaded urm.npz
Loaded urm_train.npz
Loaded urm_val.npz
Loaded icm.npz


### Parameters

In [3]:
output_folder_path = "tuner"

### Read already trained models

In [4]:
models = []

In [5]:
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

model = ItemKNNCFRecommender(URM)
model.load_model(
    output_folder_path, 
    file_name=model.RECOMMENDER_NAME + "_best_model_last.zip"
)
models.append(model)

ItemKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
ItemKNNCFRecommender: Loading model from file 'result_experimentsItemKNNCFRecommender_best_model_last.zip'
ItemKNNCFRecommender: Loading complete


In [6]:
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

model = P3alphaRecommender(URM)
model.load_model(
    output_folder_path, 
    file_name=model.RECOMMENDER_NAME + "_best_model_last.zip"
)
models.append(model)

P3alphaRecommender: URM Detected 3461 (12.4%) items with no interactions.
P3alphaRecommender: Loading model from file 'result_experimentsP3alphaRecommender_best_model_last.zip'
P3alphaRecommender: Loading complete


### Generate hybrid model

In [7]:
from src.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 759 ( 1.8%) Users that have less than 1 test interactions


In [8]:
alpha = 0.4
new_similarity = (1 - alpha) * models[0].W_sparse + alpha * models[1].W_sparse
new_similarity

<27968x27968 sparse matrix of type '<class 'numpy.float32'>'
	with 13648689 stored elements in Compressed Sparse Row format>

In [9]:
from src.Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

final_model = ItemKNNCustomSimilarityRecommender(URM_train)
final_model.fit(new_similarity)

ItemKNNCustomSimilarityRecommender: URM Detected 3461 (12.4%) items with no interactions.


### Results

In [10]:
result_df, _ = evaluator_validation.evaluateRecommender(final_model)
result_df

EvaluatorHoldout: Processed 40870 (100.0%) in 1.35 min. Users per second: 504


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.104426,0.231693,0.224226,0.062769,0.143859,0.3218,0.211702,0.142492,0.563739,0.44436,...,0.981768,0.55346,0.981768,0.049058,8.818314,0.990836,0.111151,0.653713,4.464604,0.327226


### Save recommendations

In [11]:
from utilities import save_recommendations

save_recommendations(final_model)

Saving recommendations finished!                                                                    
